In [1]:

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [2]:
# ========= Device =========
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# ========= Data Preparation =========
df = pd.read_csv('Train_timeseries_filled.csv')
df = df.loc[:100_000]


class Seq2SeqDataset(Dataset):
    def __init__(self, data, hist_len=240, fut_len=240):
        self.X = torch.tensor(data.values, dtype=torch.float32)
        self.hist, self.fut = hist_len, fut_len

    def __len__(self):
        return len(self.X) - self.hist - self.fut + 1

    def __getitem__(self, idx):
        x = self.X[idx:idx+self.hist]              # (hist, 6)
        y = self.X[idx+self.hist:idx+self.hist+self.fut]  # (fut, 6)
        return x, y

hist_len, fut_len = 240, 240
dataset = Seq2SeqDataset(df, hist_len, fut_len)

# 80/20 split
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_ds, test_ds = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=False, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=64, shuffle=False, pin_memory=True)

Using device: cuda


In [3]:
# ========= Model =========
class RNNSeq2Seq(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64, num_layers=2, fut_len=240):
        super().__init__()
        self.encoder = nn.RNN(input_dim, hidden_dim, num_layers,
                              batch_first=True, bidirectional=False)
        self.decoder = nn.RNN(input_dim, hidden_dim, num_layers,
                              batch_first=True, bidirectional=False)
        self.proj = nn.Linear(hidden_dim, input_dim)
        self.fut_len = fut_len

    def forward(self, src):
        # src: (B, hist_len, 6)
        _, h = self.encoder(src)  # h: (num_layers, B, hidden_dim)
        # start decoder with last observed value
        dec_input = src[:, -1:, :]  # (B, 1, 6)
        outputs = []
        for _ in range(self.fut_len):
            out, h = self.decoder(dec_input, h)
            val = self.proj(out)          # (B, 1, 6)
            outputs.append(val)
            dec_input = val               # feed back
        return torch.cat(outputs, dim=1)  # (B, fut_len, 6)

model = RNNSeq2Seq(input_dim=6, hidden_dim=64, num_layers=2, fut_len=fut_len)
model = model.to(device)

# ========= Training & Evaluation =========
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()


In [4]:
def train_epoch():
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def eval_epoch():
    model.eval()
    preds, truths = [], []
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds.append(out.cpu().numpy().reshape(-1,6))
        truths.append(yb.cpu().numpy().reshape(-1,6))
    preds = np.vstack(preds)
    truths = np.vstack(truths)
    return (
        mean_squared_error(truths, preds),
        mean_absolute_error(truths, preds),
        r2_score(truths, preds)
    )



In [5]:
epochs = 10
for ep in range(1, epochs+1):
    train_loss = train_epoch()
    mse, mae, r2 = eval_epoch()
    print(f"Epoch {ep}/{epochs} | Train MSE Loss: {train_loss:.4f} | "
          f"Test MSE: {mse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f}")

Epoch 1/10 | Train MSE Loss: 1.7121 | Test MSE: 1.6904 | MAE: 0.9811 | R2: 0.2102
Epoch 2/10 | Train MSE Loss: 1.7518 | Test MSE: 1.9032 | MAE: 1.0643 | R2: 0.1119
Epoch 3/10 | Train MSE Loss: 1.8217 | Test MSE: 1.7683 | MAE: 1.0091 | R2: 0.1732
Epoch 4/10 | Train MSE Loss: 1.7364 | Test MSE: 1.7664 | MAE: 1.0091 | R2: 0.1742
Epoch 5/10 | Train MSE Loss: 1.7326 | Test MSE: 1.7021 | MAE: 0.9865 | R2: 0.2035
Epoch 6/10 | Train MSE Loss: 1.6971 | Test MSE: 1.7021 | MAE: 0.9850 | R2: 0.2043
Epoch 7/10 | Train MSE Loss: 1.7117 | Test MSE: 1.7126 | MAE: 0.9891 | R2: 0.1994
Epoch 8/10 | Train MSE Loss: 1.6981 | Test MSE: 1.6983 | MAE: 0.9817 | R2: 0.2060
Epoch 9/10 | Train MSE Loss: 1.7347 | Test MSE: 1.8014 | MAE: 1.0268 | R2: 0.1581
Epoch 10/10 | Train MSE Loss: 1.6324 | Test MSE: 1.5345 | MAE: 0.9149 | R2: 0.2816
